# Real-time Implementation and Automation with FXCM 

---------------------------------------------

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with an FXCM Practice/Demo Account and NOT with a Live Trading Account._

----------------------------------------------------

## Recap: Historical Data, real-time Data and Orders

In [1]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
api.get_candles("EUR/USD", start = "2020-07-01", end = "2020-07-31",  period = "D1")

In [ ]:
api.get_candles("EUR/USD", number = 10, period = "m1")

In [ ]:
api.subscribe_market_data("EUR/USD")

In [ ]:
api.get_subscribed_symbols()

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))

In [ ]:
api.subscribe_market_data("EUR/USD", (print_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
api.get_open_positions()

In [ ]:
api.create_market_buy_order("EUR/USD", 100)

In [ ]:
order = api.create_market_buy_order("EUR/USD", 100)

In [ ]:
order.get_time()

In [ ]:
order.get_isBuy()

In [ ]:
order.get_currency()

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
api.get_open_positions()[col]

In [ ]:
api.create_market_sell_order("EUR/USD", 200)

In [ ]:
api.get_open_positions()[col]

In [ ]:
api.close_all_for_symbol("EUR/USD")

In [ ]:
api.get_open_positions()

In [ ]:
api.get_closed_positions()[col]

In [ ]:
api.get_closed_positions_summary()[col]

In [ ]:
api.get_accounts().T

In [ ]:
api.close()

## Preview: A Trader Class live in action

In [1]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime

In [2]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [3]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [4]:
class ConTrader():
    
    def __init__(self, instrument, bar_length, window, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):  
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

__Simple Contrarian: Bar_lenght = 1min | Window = 1 (1 minute)__

In [5]:
trader = ConTrader("EUR/USD", bar_length = "1min", window = 1, units = 100)

In [6]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 

Exception in thread Thread-1076:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664825340000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-03 19:29:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\User

24 

Exception in thread Thread-1091:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664825340000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-03 19:29:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\User

25 

...

104 

Exception in thread Thread-1981:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664825340000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-03 19:29:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\User

105 

Exception in thread Thread-1994:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664825340000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-03 19:29:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\User

106 

In [7]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0:
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

Exception in thread Thread-2023:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664825340000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-03 19:29:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\User

107 

Exception in thread Thread-2027:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664825340000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-03 19:29:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\User

108 

__Simple Contrarian: Bar_lenght = 1min | Window = 60 (1 hour)__

In [8]:
trader = ConTrader("EUR/USD", bar_length = "1min", window = 60, units = 100)

In [10]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

1 2 3 4 5 6 7 8 9 10 11 12 13 

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\336370571.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length,


14 15 16 17 18 19 20 
----------------------------------------------------------------------------------------------------
2022-10-03 19:33:02.170000 | GOING SHORT
2022-10-03 19:33:02.170000 | units = 100 | price = 0.98187 | Unreal. P&L = -3
----------------------------------------------------------------------------------------------------

21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\336370571.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length,


102 103 104 105 106 107 108 109 110 

In [11]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0:
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0



2022-10-03 19:34:11.336856 | GOING NEUTRAL

  tradeId  amountK currency  grossPL  isBuy
0              500  EUR/USD      -46  False


In [12]:
api.close()

## Collecting and storing real-time tick data

In [13]:
import pandas as pd
import fxcmpy

In [14]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [15]:
def print_data(data, dataframe):
    print(dataframe)

In [17]:
api.subscribe_market_data("EUR/USD", (print_data, ))

In [18]:
api.unsubscribe_market_data("EUR/USD")

                             Bid      Ask     High      Low
2022-10-03 19:34:57.812  0.98177  0.98186  0.98449  0.97523
2022-10-03 19:34:57.998  0.98177  0.98186  0.98449  0.97523


In [19]:
tick_data = None

In [20]:
def get_tick_data(data, dataframe):
    
    global tick_data # global variable tick_data
    
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))
    
    tick_data = dataframe.iloc[:, :2]

In [21]:
api.subscribe_market_data("EUR/USD", (get_tick_data, ))

  2 | EUR/USD | 2022-10-03 19:35:08.719000, 0.98174, 0.98181
  3 | EUR/USD | 2022-10-03 19:35:10.249000, 0.9817, 0.98177
  4 | EUR/USD | 2022-10-03 19:35:10.409000, 0.9817, 0.98178
  5 | EUR/USD | 2022-10-03 19:35:10.973000, 0.98171, 0.98179
  6 | EUR/USD | 2022-10-03 19:35:11.765000, 0.98171, 0.98179
  7 | EUR/USD | 2022-10-03 19:35:12.576000, 0.9817, 0.98178
  8 | EUR/USD | 2022-10-03 19:35:14.723000, 0.98177, 0.98187
  9 | EUR/USD | 2022-10-03 19:35:15.226000, 0.9818, 0.9818800000000002
 10 | EUR/USD | 2022-10-03 19:35:16.153000, 0.9818, 0.98189
 11 | EUR/USD | 2022-10-03 19:35:16.722000, 0.98181, 0.98191
 12 | EUR/USD | 2022-10-03 19:35:17.919000, 0.98181, 0.9818800000000002
 13 | EUR/USD | 2022-10-03 19:35:19.011000, 0.98181, 0.98189
 14 | EUR/USD | 2022-10-03 19:35:20.235000, 0.98179, 0.98186
 15 | EUR/USD | 2022-10-03 19:35:20.956000, 0.98185, 0.98195
 16 | EUR/USD | 2022-10-03 19:35:22.168000, 0.98186, 0.98196
 17 | EUR/USD | 2022-10-03 19:35:22.718000, 0.98185, 0.98194
 18 | E

In [22]:
api.unsubscribe_market_data("EUR/USD")

 23 | EUR/USD | 2022-10-03 19:35:26.616000, 0.98186, 0.98195
 24 | EUR/USD | 2022-10-03 19:35:27.640000, 0.98186, 0.98194


In [23]:
tick_data

,Bid,Ask
2022-10-03 19:35:05.745,0.98171,0.98187
2022-10-03 19:35:08.719,0.98174,0.98181
2022-10-03 19:35:10.249,0.98170,0.98177
2022-10-03 19:35:10.409,0.98170,0.98178
2022-10-03 19:35:10.973,0.98171,0.98179
2022-10-03 19:35:11.765,0.98171,0.98179
2022-10-03 19:35:12.576,0.98170,0.98178
2022-10-03 19:35:14.723,0.98177,0.98187
2022-10-03 19:35:15.226,0.98180,0.98188
2022-10-03 19:35:16.153,0.98180,0.98189


In [24]:
tick_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2022-10-03 19:35:05.745000 to 2022-10-03 19:35:27.640000
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Bid     24 non-null     float64
 1   Ask     24 non-null     float64
dtypes: float64(2)
memory usage: 576.0 bytes


In [25]:
api.close()

## Storing and resampling real-time tick data (Part 1)

In [26]:
import pandas as pd
import fxcmpy

In [27]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

In [28]:
tick_data = None
resamp = None
bar_length = "10s"

In [29]:
def get_tick_data(data, dataframe):
    
    global tick_data, resamp # global variables
    
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))
    
    tick_data = dataframe.iloc[:, :2]
    resamp = tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1]
    resamp["Mid"] = (resamp.Ask + resamp.Bid)/2

In [30]:
api.subscribe_market_data("EUR/USD", (get_tick_data, ))

  2 | EUR/USD | 2022-10-03 19:39:58.594000, 0.98228, 0.98234
  3 | EUR/USD | 2022-10-03 19:39:58.979000, 0.98227, 0.98233
  4 | EUR/USD | 2022-10-03 19:39:59.710000, 0.98228, 0.98234
  5 | EUR/USD | 2022-10-03 19:40:00.053000, 0.98227, 0.98235
  6 | EUR/USD | 2022-10-03 19:40:00.840000, 0.98224, 0.98231
  7 | EUR/USD | 2022-10-03 19:40:01.400000, 0.98226, 0.98233
  8 | EUR/USD | 2022-10-03 19:40:01.696000, 0.98226, 0.9823200000000002
  9 | EUR/USD | 2022-10-03 19:40:02.952000, 0.98222, 0.98229
 10 | EUR/USD | 2022-10-03 19:40:03.462000, 0.98222, 0.98229
 11 | EUR/USD | 2022-10-03 19:40:05.019000, 0.98223, 0.98231
 12 | EUR/USD | 2022-10-03 19:40:05.790000, 0.98223, 0.98229
 13 | EUR/USD | 2022-10-03 19:40:06.622000, 0.98224, 0.98231
 14 | EUR/USD | 2022-10-03 19:40:07.108000, 0.98227, 0.98233
 15 | EUR/USD | 2022-10-03 19:40:07.588000, 0.98227, 0.98233
 16 | EUR/USD | 2022-10-03 19:40:08.345000, 0.98225, 0.98231
 17 | EUR/USD | 2022-10-03 19:40:08.770000, 0.98224, 0.98231
 18 | EUR/USD

In [31]:
api.unsubscribe_market_data("EUR/USD")

 25 | EUR/USD | 2022-10-03 19:40:15.873000, 0.98222, 0.98228
 26 | EUR/USD | 2022-10-03 19:40:16.337000, 0.98222, 0.98229
 27 | EUR/USD | 2022-10-03 19:40:17.076000, 0.98224, 0.9823


In [32]:
tick_data

,Bid,Ask
2022-10-03 19:39:58.327,0.98228,0.98235
2022-10-03 19:39:58.594,0.98228,0.98234
2022-10-03 19:39:58.979,0.98227,0.98233
2022-10-03 19:39:59.710,0.98228,0.98234
2022-10-03 19:40:00.053,0.98227,0.98235
2022-10-03 19:40:00.840,0.98224,0.98231
2022-10-03 19:40:01.400,0.98226,0.98233
2022-10-03 19:40:01.696,0.98226,0.98232
2022-10-03 19:40:02.952,0.98222,0.98229
2022-10-03 19:40:03.462,0.98222,0.98229


In [33]:
resamp

,Bid,Ask,Mid
2022-10-03 19:40:00,0.98228,0.98234,0.982310
2022-10-03 19:40:10,0.98224,0.98231,0.982275


In [34]:
api.close()

## A Contrader Class

In [35]:
import pandas as pd
import fxcmpy

In [36]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

In [37]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = bar_length
        self.tick_data = None
        self.data = None
        self.ticks = 0 # running Tick number
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ") # Print running Tick number
        
        # collect and store tick data (mid-price only)
        self.tick_data = dataframe.iloc[:, :2].copy()
        self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
        self.tick_data = self.tick_data[self.instrument].to_frame()
        
        self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]

In [38]:
for i in range(10):
    print(i, end = " ")

0 1 2 3 4 5 6 7 8 9 

In [39]:
trader = ConTrader("EUR/USD", "10s")

In [40]:
trader

In [41]:
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 

In [42]:
api.unsubscribe_market_data("EUR/USD")

103 

In [43]:
trader.data

,EUR/USD
2022-10-03 19:44:00,0.982055
2022-10-03 19:44:10,0.981975
2022-10-03 19:44:20,0.982090
2022-10-03 19:44:30,0.982135
2022-10-03 19:44:40,0.982145
2022-10-03 19:44:50,0.982060
2022-10-03 19:45:00,0.981940
2022-10-03 19:45:10,0.982155


In [44]:
trader.tick_data

,EUR/USD
2022-10-03 19:43:52.170,0.982110
2022-10-03 19:43:52.167,0.982110
2022-10-03 19:43:52.882,0.982080
2022-10-03 19:43:53.222,0.982090
2022-10-03 19:43:53.780,0.982100
...,...
2022-10-03 19:45:15.181,0.982230
2022-10-03 19:45:15.957,0.982235
2022-10-03 19:45:16.367,0.982230
2022-10-03 19:45:17.020,0.982230


In [45]:
api.close()

## Storing and resampling real-time tick data (Part 2)

__When should we resample?__<br>
Whenever there are more than 5s (__bar_length__) between the __most recent tick__ and __last full bar__. 

In [46]:
import pandas as pd
import fxcmpy
from datetime import datetime

In [47]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

In [48]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) # Pandas Timedelta Object
        self.tick_data = None
        self.data = None
        self.ticks = 0
        self.last_bar = pd.to_datetime(datetime.utcnow()) # Time at instantiation
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms") # Pandas Timestamp Object
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.iloc[:, :2].copy()
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        self.last_bar = self.data.index[-1]  # update time of last full bar

In [49]:
datetime.utcnow()

datetime.datetime(2022, 10, 3, 19, 55, 29, 304851)

In [50]:
pd.to_datetime(datetime.utcnow())

Timestamp('2022-10-03 19:55:29.320855')

In [51]:
pd.to_timedelta("10s")

Timedelta('0 days 00:00:10')

In [52]:
trader = ConTrader("EUR/USD", "10s")
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

1 2 3 4 5 6 7 8 9 10 11 12 

Exception in thread Thread-11397:
Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Bongo\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Bongo\anaconda3\lib\site-packages\socketio\client.py", line 615, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "C:\Users\Bongo\anaconda3\lib\site-packages\socketio\client.py", line 501, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "C:\Users\Bongo\anaconda3\lib\site-packages\socketio\client.py", line 547, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "C:\Users\Bongo\anaconda3\lib\site-packages\fxcmpy\fxcmpy.py", line 2555, in __on_price_update__
    callbacks[func](data, self.prices[symbol])
  File "C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\3038864429.py", line 22, in get_tick_data

13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 

In [53]:
api.unsubscribe_market_data("EUR/USD")

52 

In [54]:
trader.data

,EUR/USD
2022-10-03 19:55:40,0.982185
2022-10-03 19:55:50,0.982220
2022-10-03 19:56:00,0.982200


In [55]:
trader.tick_data

,EUR/USD
2022-10-03 19:55:30.515,0.982195
2022-10-03 19:55:30.515,0.982195
2022-10-03 19:55:31.047,0.982195
2022-10-03 19:55:31.522,0.982190
2022-10-03 19:55:32.286,0.982195
2022-10-03 19:55:32.672,0.982230
2022-10-03 19:55:33.366,0.982230
2022-10-03 19:55:34.352,0.982220
2022-10-03 19:55:34.991,0.982230
2022-10-03 19:55:37.183,0.982220


In [56]:
api.close()

## Storing and resampling real-time tick data (Part 3)

In [ ]:
import pandas as pd
import fxcmpy
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

Goal: Only use/store the latests ticks since the (second) last full bar.

In [ ]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.data = pd.DataFrame() # NEW
        self.ticks = 0
        self.last_bar = pd.to_datetime(datetime.utcnow()) 
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            #self.tick_data = dataframe.iloc[:, :2].copy()
            # NEW: get most recent ticks since last full bar
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            
    def resample_and_join(self):
        #self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        # NEW: append the most recent ticks (resampled) to self.data
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.data.index[-1]  

In [ ]:
trader = ConTrader("EUR/USD", "10s")
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
trader.data

In [ ]:
trader.tick_data

In [ ]:
api.close()

## Working with historical data and real-time tick data (Part 1)

In [58]:
import pandas as pd
import fxcmpy

In [59]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [60]:
instrument = "EUR/USD"

In [61]:
df = api.get_candles(instrument, number = 250, period = "m1", columns = ["bidclose", "askclose"])
df

,bidclose,askclose
date,,
2022-10-03 15:50:00,0.98206,0.98213
2022-10-03 15:51:00,0.98191,0.98198
2022-10-03 15:52:00,0.98216,0.98222
2022-10-03 15:53:00,0.98198,0.98205
2022-10-03 15:54:00,0.98190,0.98198
...,...,...
2022-10-03 19:55:00,0.98217,0.98223
2022-10-03 19:56:00,0.98218,0.98226
2022-10-03 19:57:00,0.98245,0.98252


In [62]:
df[instrument] = (df.bidclose + df.askclose) / 2

In [63]:
df

,bidclose,askclose,EUR/USD
date,,,
2022-10-03 15:50:00,0.98206,0.98213,0.982095
2022-10-03 15:51:00,0.98191,0.98198,0.981945
2022-10-03 15:52:00,0.98216,0.98222,0.982190
2022-10-03 15:53:00,0.98198,0.98205,0.982015
2022-10-03 15:54:00,0.98190,0.98198,0.981940
...,...,...,...
2022-10-03 19:55:00,0.98217,0.98223,0.982200
2022-10-03 19:56:00,0.98218,0.98226,0.982220
2022-10-03 19:57:00,0.98245,0.98252,0.982485


In [64]:
df = df[instrument].to_frame()

In [65]:
df = df.resample("20min", label = "right").last().dropna().iloc[:-1]
df

,EUR/USD
date,
2022-10-03 16:00:00,0.980820
2022-10-03 16:20:00,0.980310
2022-10-03 16:40:00,0.980435
2022-10-03 17:00:00,0.980790
2022-10-03 17:20:00,0.979880
2022-10-03 17:40:00,0.979785
2022-10-03 18:00:00,0.979445
2022-10-03 18:20:00,0.980325
2022-10-03 18:40:00,0.981820


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12 entries, 2022-10-03 16:00:00 to 2022-10-03 19:40:00
Freq: 20T
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   EUR/USD  12 non-null     float64
dtypes: float64(1)
memory usage: 192.0 bytes


In [73]:
api.close()

## Working with historical data and real-time tick data (Part 2)

In [71]:
import pandas as pd
import fxcmpy
from datetime import datetime

In [74]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [75]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.data = None # first defined in get_most_recent()
        self.ticks = 0
        self.last_bar = None # first defined in get_most_recent()
    
    def get_most_recent(self, period = "m1", number = 10000): # NEW
        df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
        df[self.instrument] = (df.bidclose + df.askclose) / 2
        df = df[self.instrument].to_frame()
        df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
        self.data = df.copy() # first defined
        self.last_bar = self.data.index[-1] # first defined
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.data.index[-1]  

In [76]:
trader = ConTrader("EUR/USD", "1min")

In [79]:
print(datetime.utcnow())
trader.get_most_recent(period = "m1", number = 10000)
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

2022-10-03 20:02:58.098922
2 3 4 5 

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,
C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,
C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,
C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

6 

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,


7 

...

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,


41 

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,


42 

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,


In [80]:
api.unsubscribe_market_data("EUR/USD")

43 

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,


44 

C:\Users\Bongo\AppData\Local\Temp\ipykernel_15100\787887241.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(self.tick_data.resample(self.bar_length,


In [81]:
trader.data.tail(10)

,EUR/USD
2022-10-03 19:53:00,0.982250
2022-10-03 19:54:00,0.982200
2022-10-03 19:55:00,0.982330
2022-10-03 19:56:00,0.982200
2022-10-03 19:57:00,0.982220
2022-10-03 19:58:00,0.982485
2022-10-03 19:59:00,0.982520
2022-10-03 20:00:00,0.982870
2022-10-03 20:01:00,0.982675
2022-10-03 20:02:00,0.982635


In [82]:
api.close()

## Working with historical data and real-time tick data (Part 3)

In [83]:
import pandas as pd
import fxcmpy
from datetime import datetime
import time

In [84]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [85]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        
        self.count = 0 # for demonstration purposes only
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True: # repeat until we get all historical bars
            self.count += 1 # for demonstration purposes only
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.data = df.copy()
            self.last_bar = self.data.index[-1]
            # accept, if less than [bar_lenght] has elapsed since the last full historical bar and now
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.data.index[-1]  

In [86]:
trader = ConTrader("EUR/USD", "1min")

In [87]:
print(datetime.utcnow())
trader.get_most_recent()
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

2022-10-04 14:55:18.306540
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 

Exception in thread Thread-28261:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664895300000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-04 14:55:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Use

17 

Exception in thread Thread-28276:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664895300000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-04 14:55:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Use

18 

...

25 

Exception in thread Thread-28371:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664895300000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-04 14:55:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Use

26 

Exception in thread Thread-28380:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664895300000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-04 14:55:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Use

27 

In [88]:
api.unsubscribe_market_data("EUR/USD")

Exception in thread Thread-28391:
Traceback (most recent call last):
  File "pandas\_libs\index.pyx", line 545, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1664895300000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Bongo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 513, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas\_libs\index.pyx", line 547, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2022-10-04 14:55:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Use

28 

In [89]:
trader.data.tail(10)

,EUR/USD
date,
2022-10-04 14:46:00,0.994195
2022-10-04 14:47:00,0.993950
2022-10-04 14:48:00,0.994485
2022-10-04 14:49:00,0.994860
2022-10-04 14:50:00,0.994295
2022-10-04 14:51:00,0.994045
2022-10-04 14:52:00,0.993940
2022-10-04 14:53:00,0.994170
2022-10-04 14:54:00,0.994400


In [90]:
trader.tick_data

In [91]:
trader.count

2

In [92]:
api.close()

## Defining a Simple Contrarian Trading Strategy

In [93]:
import pandas as pd
import numpy as np
import fxcmpy
from datetime import datetime
import time

In [94]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [95]:
class ConTrader():
    def __init__(self, instrument, bar_length, window):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None # NEW
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy() # raw!
            self.last_bar = self.raw_data.index[-1] # raw!
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() # Prepare Data / Strategy Features
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1]) # raw!
        self.last_bar = self.raw_data.index[-1] # raw!  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy() # self.raw_data new!
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()  # first defined here
        

In [96]:
trader = ConTrader("EUR/USD", "1min", window = 1)

In [98]:
trader.get_most_recent()
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [99]:
api.unsubscribe_market_data("EUR/USD")

1 

In [100]:
trader.raw_data.tail(10)

,EUR/USD
date,
2022-10-04 14:49:00,0.994860
2022-10-04 14:50:00,0.994295
2022-10-04 14:51:00,0.994045
2022-10-04 14:52:00,0.993940
2022-10-04 14:53:00,0.994170
2022-10-04 14:54:00,0.994400
2022-10-04 14:55:00,0.994880
2022-10-04 14:56:00,0.994925
2022-10-04 14:57:00,0.995330


In [101]:
trader.data.tail(10)

AttributeError: 'NoneType' object has no attribute 'tail'

In [105]:
api.close()

## Placing Orders and Executing Trades

In [106]:
import pandas as pd
import numpy as np
import fxcmpy
from datetime import datetime
import time


In [107]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

ServerError: Can not find FXCM Server.

packet queue is empty, aborting


__Assumption: Netting Account and NOT Hedging Account (Practice Account: only Hedging is available)__ <br>
(Consequence for Trading Sessions with Practice Account: Double Costs)

In [ ]:
# Assumption: Netting Account and NOT Hedging Account (Practice Account: only Hedging is available)

class ConTrader():
    def __init__(self, instrument, bar_length, window, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units # NEW
        self.position = 0 # NEW
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades() # NEW!
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self): # NEW!
        if self.data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                print("GOING LONG")
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                print("GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: # if position is short -> go/stay short 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                print("GOING SHORT")
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                print("GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                print("GOING NEUTRAL")
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                print("GOING NEUTRAL")
            self.position = 0
        

In [ ]:
trader = ConTrader("EUR/USD", "1min", window = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
api.get_open_positions()#[col]

In [ ]:
api.close_all_for_symbol("EUR/USD")

In [ ]:
api.get_closed_positions()[col]

In [ ]:
api.get_closed_positions_summary()[col]

In [ ]:
trader.data.tail(10)

In [ ]:
api.close()

## Trade Monitoring and Reporting

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
class ConTrader():
    
    def __init__(self, instrument, bar_length, window, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  # NEW
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  # NEW
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  # NEW
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0

    def report_trade(self, order, going):  # NEW
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
trader = ConTrader("EUR/USD", "1min", window = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
trader.data.tail(12)

In [ ]:
api.close()

## Trading other Strategies - Coding Challenge

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

__Strategy 1__: SMA 50/200 (minutes) Crossover (200 ticks)

## Stop here if you don´t want to see the solution!

###############################################################

In [ ]:
class SMATrader():
    
    def __init__(self, instrument, bar_length, SMA_S, SMA_L, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.SMA_S = SMA_S
        self.SMA_L = SMA_L
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA_S"] = df[self.instrument].rolling(self.SMA_S).mean()
        df["SMA_L"] = df[self.instrument].rolling(self.SMA_L).mean()
        df["position"] = np.where(df["SMA_S"] > df["SMA_L"], 1, -1)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG") 
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG") 
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
trader = SMATrader("EUR/USD", "1min", SMA_S = 50, SMA_L = 200, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
trader.data.tail(10)

In [ ]:
trader.data.tail(30).plot(figsize = (12, 8), secondary_y = "position")
plt.show()

__Strategy 2__: Bollinger Bands SMA 20 (minutes) / 1 Standard Deviation (200 ticks) 

## Stop here if you don´t want to see the solution!

###############################################################

In [ ]:
class BollTrader():
    
    def __init__(self, instrument, bar_length, SMA, dev, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.SMA = SMA
        self.dev = dev
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA"] = df[self.instrument].rolling(self.SMA).mean()
        df["Lower"] = df["SMA"] - df[self.instrument].rolling(self.SMA).std() * self.dev
        df["Upper"] = df["SMA"] + df[self.instrument].rolling(self.SMA).std() * self.dev
        df["distance"] = df[self.instrument] - df.SMA
        df["position"] = np.where(df[self.instrument] < df.Lower, 1, np.nan)
        df["position"] = np.where(df[self.instrument] > df.Upper, -1, df["position"])
        df["position"] = np.where(df.distance * df.distance.shift(1) < 0, 0, df["position"])
        df["position"] = df.position.ffill().fillna(0)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
trader = BollTrader("EUR/USD", "1min", SMA = 20, dev = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
trader.data

In [ ]:
trader.data.tail(20)[["EUR/USD", "SMA", "Lower", "Upper"]].plot(figsize = (12, 8))
plt.show()

In [ ]:
api.close()

## Machine Learning Strategies (1) - Model Fitting

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [ ]:
data = pd.read_csv("five_minute.csv", parse_dates = ["time"], index_col = "time")

In [ ]:
data

In [ ]:
data["returns"] = np.log(data.div(data.shift(1)))

In [ ]:
data.dropna(inplace = True)

In [ ]:
data["direction"] = np.sign(data.returns)

In [ ]:
data

In [ ]:
lags = 2

In [ ]:
cols = []
for lag in range(1, lags + 1):
    col = "lag{}".format(lag)
    data[col] = data.returns.shift(lag)
    cols.append(col)
data.dropna(inplace = True)

In [ ]:
data

In [ ]:
lm = LogisticRegression(C = 1e6, max_iter = 100000, multi_class = "ovr")

In [ ]:
lm.fit(data[cols], data.direction)

In [ ]:
lm

__In-Sample "Prediction"__

In [ ]:
data["pred"] = lm.predict(data[cols])

In [ ]:
data

In [ ]:
hits = np.sign(data.direction * data.pred).value_counts()

In [ ]:
hits

In [ ]:
hit_ratio = hits[1.0] / sum(hits)
hit_ratio

In [ ]:
lm

__Saving the model__

In [ ]:
import pickle

In [ ]:
pickle.dump(lm, open("logreg.pkl", "wb"))

## Machine Learning Strategies (2) - Implementation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import fxcmpy
import time
from datetime import datetime
import pickle

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
class MLTrader():
    
    def __init__(self, instrument, bar_length, lags, model, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.lags = lags
        self.model = model
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df = df.append(self.tick_data.iloc[-1]) # append latest tick (== open price of current bar)
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        cols = []
        for lag in range(1, self.lags + 1):
            col = "lag{}".format(lag)
            df[col] = df.returns.shift(lag)
            cols.append(col)
        df.dropna(inplace = True)
        df["position"] = lm.predict(df[cols])
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
lm = pickle.load(open("logreg.pkl", "rb"))

In [ ]:
lm

In [ ]:
trader = MLTrader("EUR/USD", "5min", lags = 2, model = lm, units = 100)

In [ ]:
trader.model

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
trader.data.tail(10)

In [ ]:
api.close()

##########################################################################################################

(you can find the corresponding Video for the following lines of code in __Section 23__: Cloud Deployment (AWS) | Scheduling Trading Sessions | Full Automation)

## How to Time/Schedule the End of a Trading Session (Simple Contrarian)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
datetime.utcnow().time() # current utc time

In [ ]:
pd.to_datetime("17:30").time() # desired end of trading session (in utc time)

In [ ]:
if datetime.utcnow().time() >= pd.to_datetime("17:30").time():
    print("Stop the Trading Session!")

In [ ]:
class ConTrader():
    
    def __init__(self, instrument, bar_length, window, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # define stop
        if recent_tick.time() >= pd.to_datetime("09:27").time():
            api.unsubscribe_market_data(self.instrument)
            if len(api.get_open_positions()) != 0:
                api.close_all_for_symbol(self.instrument)
                print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
                time.sleep(20)
                print(api.get_closed_positions_summary()[col])
                self.position = 0
                api.close()
        
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):  
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
trader = ConTrader("EUR/USD", bar_length = "1min", window = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.close()